In [2]:
from datasets import load_dataset
#['adversarialQA', 'dbidaf', 'dbert', 'droberta']
dataset1 = load_dataset("adversarial_qa",'adversarialQA')
dataset2 = load_dataset("adversarial_qa",'dbidaf')
dataset3 = load_dataset("adversarial_qa",'dbert')
dataset4 = load_dataset("adversarial_qa",'droberta')

Found cached dataset adversarial_qa (/home/nlplab/.cache/huggingface/datasets/adversarial_qa/adversarialQA/1.0.0/92356be07b087c5c6a543138757828b8d61ca34de8a87807d40bbc0e6c68f04b)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset adversarial_qa (/home/nlplab/.cache/huggingface/datasets/adversarial_qa/dbidaf/1.0.0/92356be07b087c5c6a543138757828b8d61ca34de8a87807d40bbc0e6c68f04b)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset adversarial_qa (/home/nlplab/.cache/huggingface/datasets/adversarial_qa/dbert/1.0.0/92356be07b087c5c6a543138757828b8d61ca34de8a87807d40bbc0e6c68f04b)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset adversarial_qa (/home/nlplab/.cache/huggingface/datasets/adversarial_qa/droberta/1.0.0/92356be07b087c5c6a543138757828b8d61ca34de8a87807d40bbc0e6c68f04b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train_dataset = [dataset1['train'],dataset2['train'],dataset3['train'],dataset4['train']]
test_dataset = [dataset1['test'],dataset2['test'],dataset3['test'],dataset4['test']]
val_dataset = [dataset1['validation'],dataset2['validation'],
               dataset3['validation'],dataset4['validation']]

In [8]:
dataset1['train'][0]

{'id': '7ba1e8f4261d3170fcf42e84a81dd749116fae95',
 'title': 'Brain',
 'context': 'Another approach to brain function is to examine the consequences of damage to specific brain areas. Even though it is protected by the skull and meninges, surrounded by cerebrospinal fluid, and isolated from the bloodstream by the blood–brain barrier, the delicate nature of the brain makes it vulnerable to numerous diseases and several types of damage. In humans, the effects of strokes and other types of brain damage have been a key source of information about brain function. Because there is no ability to experimentally control the nature of the damage, however, this information is often difficult to interpret. In animal studies, most commonly involving rats, it is possible to use electrodes or locally injected chemicals to produce precise patterns of damage and then examine the consequences for behavior.',
 'question': 'What sare the benifts of the blood brain barrir?',
 'answers': {'text': ['isolated

In [1]:
import json
import sys
import random
sys.path.append('../')

In [2]:
from src.dataset_processor import QuestionGenerationData
from src.config import GenerationTasks
from nltk.tokenize import sent_tokenize
from nltk.util import ngrams
def selectContext(articles, answer_index,n=4):
    vals = list(ngrams(range(len(articles)),n))
    for l in vals:
        if answer_index in l:
            return l

def get_span_sentence(span_answer,passage,n=4):
    answer = span_answer.strip()
    context = passage.strip()
    contexts = sent_tokenize(context)
    
    answer_index = [idx for idx, s in enumerate(contexts) if answer in s]
    answer_sentences = [s for idx, s in enumerate(contexts) if answer in s]

    if len(contexts) < 5 and len(answer_index) >0:
        return passage.strip(), " ".join(answer_sentences)
    if len(answer_index) < 1:
        return None
    else:
        if len(answer_index)==1:
            selected_context_idxs = selectContext(contexts, answer_index, n=n)
            if selected_context_idxs is not None:
                answer_context = ' '.join([contexts[sid] for sid in selected_context_idxs])
            else:
                answer_context = passage.strip()
        else:
            answer_context =  " ".join(answer_sentences)
    return answer_context,  " ".join(answer_sentences)

In [7]:
def process_adversarial_qa(data_pack):
    dataset =[]
    for pack in data_pack:
        for item in pack:
            context= item['context']
            question = item['question']
            answer = item['answers']
            if len(answer['text'])>0:
                answer_text = answer['text'][0]
                answer_start = answer['answer_start'][0]
                
                obj = get_span_sentence(answer_text,context,n=4)
                
                question_based_on_context = QuestionGenerationData(task=GenerationTasks.context_question_gen,
                                            input_text= context.strip(),
                                            output_text=question.strip(),
                                            contextual_text= answer_text.strip())
                dataset.append(question_based_on_context)
                
                if obj is not None:
                        #print(answer_sentence)
                        fact, answer_sentence = obj

                        question_based_without_context = QuestionGenerationData(task=GenerationTasks.vanilla_question_gen, 
                                                                                input_text= fact, output_text=question,
                                                                                contextual_text= answer_sentence.strip())
                        dataset.append(question_based_without_context)
            else:
                question_based_on_context = QuestionGenerationData(task=GenerationTasks.vanilla_question_gen,
                                            input_text= context.strip(),
                                            output_text=question.strip(),
                                            contextual_text= "")
                dataset.append(question_based_on_context)
    return dataset         

In [42]:
test_dataset[0][0]

{'id': 'fa2b68a340fa4292aa491e02a1498e26798c8402',
 'title': 'Civil_disobedience',
 'context': 'There have been debates as to whether civil disobedience must necessarily be non-violent. Black\'s Law Dictionary includes non-violence in its definition of civil disobedience. Christian Bay\'s encyclopedia article states that civil disobedience requires "carefully chosen and legitimate means," but holds that they do not have to be non-violent. It has been argued that, while both civil disobedience and civil rebellion are justified by appeal to constitutional defects, rebellion is much more destructive; therefore, the defects justifying rebellion must be much more serious than those justifying disobedience, and if one cannot justify civil rebellion, then one cannot justify a civil disobedients\' use of force and violence and refusal to submit to arrest. Civil disobedients\' refraining from violence is also said to help preserve society\'s tolerance of civil disobedience.',
 'question': 'What

In [8]:
train_dataset_extracted = process_adversarial_qa(train_dataset) + process_adversarial_qa(test_dataset)
val_dataset_extracted = process_adversarial_qa(val_dataset)

In [44]:

test_dataset_extracted = process_adversarial_qa(test_dataset)


In [33]:
cc=[v for v in train_dataset_extracted if v.task==GenerationTasks.vanilla_question_gen]
cw=[v for v in train_dataset_extracted if v.task==GenerationTasks.context_question_gen]

In [46]:
len(train_dataset_extracted),len(test_dataset_extracted)

(119652, 6000)

In [9]:
from dataclass_csv import DataclassReader, dateformat,DataclassWriter

with open("../curated_data/adversarial_qa_train.csv", "w",encoding='utf-8') as f:
    w = DataclassWriter(f, train_dataset_extracted, QuestionGenerationData)
    w.write()

with open("../curated_data/adversarial_qa_dev.csv", "w",encoding='utf-8') as f:
    w = DataclassWriter(f, val_dataset_extracted, QuestionGenerationData)
    w.write()

In [ ]:
from datasets import load_dataset
#['new_wiki', 'nyt', 'reddit', 'amazon']
dataset_squad_shift_wiki = load_dataset("squadshifts",'new_wiki')
dataset_squad_shift_nyt = load_dataset("squadshifts",'nyt')
dataset_squad_shift_reddit = load_dataset("squadshifts",'reddit')
dataset_squad_shift_amazon = load_dataset("squadshifts",'amazon')

In [12]:
dataset_squad_shift_nyt['test'][200]

{'id': '5d700715c8e4820a9b66acd2',
 'title': 'https://www.nytimes.com/2015/02/05/us/more-college-freshmen-report-having-felt-depressed.html',
 'context': 'Suzanne Ciechalski, a freshman at St. John’s University in Queens, said technology that might appear social in nature could in fact lead to stress and feelings of depression. “I feel like people spend a lot of time on social networks trying to create this picture of who they want to be,” Ms. Ciechalski said. “Maintaining that takes a lot of effort. I feel like being a teenager or young adult, the pressure to try and make people see you’re the best is really high.” Contrary to some reports of high rates of drinking among high school students, the survey found a continued decline in college freshmen reporting those behaviors. About one-third said they had drunk beer, wine or hard alcohol at least occasionally in the past year, compared with almost half just 10 years ago. Fewer than one in 50 students reported smoking cigarettes.',
 'qu

In [17]:
#dataset = load_dataset("cuad")
dataset = load_dataset("simple_questions_v2")

No config specified, defaulting to: simple_questions_v2/annotated


Generating train split:   0%|          | 0/75910 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/75910 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/75910 [00:00<?, ? examples/s]

Dataset simple_questions_v2 downloaded and prepared to /home/nlplab/.cache/huggingface/datasets/simple_questions_v2/annotated/1.0.0/d4192b0a3f18df90a99f9f56987766d28ae2e48bc479053894efadaf20b6e935. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
dataset['train'][10]

{'id': '10',
 'subject_entity': 'www.freebase.com/m/02dtg',
 'relationship': 'www.freebase.com/location/location/people_born_here',
 'object_entity': 'www.freebase.com/m/01s8mcb',
 'question': 'who is a musician born in detroit\n'}

In [ ]:
from datasets import load_dataset

dataset_aquamuse = load_dataset("aquamuse",'abstractive')
dataset_aquamuse_ex = load_dataset("aquamuse",'extractive')

In [21]:
dataset_aquamuse['train'][100]

{'query': 'where did the tv show the waltons take place',
 'input_urls': ['https://teeveesgreatest.webs.com/apps/videos/videos/show/18885574-the-waltons-opening-and-closing-theme-1972-1981-with-snippet-',
  'https://wikivisually.com/wiki/Ellen_Corby',
  'https://hamaika.tv/author/admin/page/88/'],
 'target': "The main story is set in Walton's Mountain, a fictional town at the foot of a mountain in fictitious Jefferson County, Virginia."}

In [6]:
dataset_aquamuse['train'][0]

{'query': 'who was the first woman selected in rajya sabha',
 'input_urls': ['https://events.artculturefestival.in/2017/11/list-bharatanatyam-dancers-india?tribe_event_display=list&tribe_paged=3',
  'https://feminisminindia.com/2018/12/05/rukmini-devi-arundale-the-dancer-who-refused-presidency-indianwomeninhistory/',
  'https://en.wikiquote.org/wiki/Rukmini_Devi_Arundale',
  'http://estrelitzia.eu/40166-the-video-descriptions-of-ancient-india/'],
 'target': "She also is the first ever woman in Indian History to be nominated as the Rajya Sabha member. She is considered the most important revivalist in the Indian classical dance form of Bharatanatyam from its original' sadhir' style, prevalent amongst the temple dancers, Devadasis, she also worked for the re-establishment of traditional Indian arts and crafts."}